In [1]:
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import clone_model
from models import get_model, resnet_v1, resnet_v2
from util import select_clean_uncertain, combine_result, inject_noise
import time
import argparse
from util import other_class
from tensorflow.python.lib.io import file_io
from keras.utils import np_utils, multi_gpu_model
from keras import backend as K
from loss_acc_plot import loss_acc_plot
from sklearn.metrics import accuracy_score
# from io import BytesIO

In [2]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
NUM_CLASSES = {'mnist': 10, 'svhn': 10, 'cifar-10': 10, 'cifar-100': 100, 'celeb': 20}
dataset = "celeb"
noise_ratio = 0
data_ratio = 100
repeat = 3
# X_train, y_train, X_test, y_test, un_selected_index = get_data('celeb', init_noise_ratio, data_ratio, random_shuffle=False)

In [4]:
def eval_model(model, test_data, test_labels):
    predictions = model.predict(test_data)
    #print(predictions)
    #predictions = list(np.around(np.array(predictions),0))
    predictions_ = np.argmax(predictions, axis = 1)
    predictions_test = np.argmax(test_labels, axis = 1)
    
    accuracy = accuracy_score(predictions_test, predictions_)
    #print(accuracy)
    return accuracy

In [5]:
def get_data(noise_ratio):
    X_train = np.load('data/image_train_20.npy')
    X_test = np.load('data/image_test_20.npy')
    y_train = np.load('data/label_train_20.npy')
    y_test = np.load('data/label_test_20.npy')

    image_shape = 128
    X_train = X_train.reshape(-1, image_shape, image_shape, 3)
    X_test = X_test.reshape(-1, image_shape, image_shape, 3)

    X_train = X_train / 255.0
    X_test = X_test / 255.0

    means = X_train.mean(axis=0)
    # std = np.std(X_train)
    X_train = (X_train - means)  # / std
    X_test = (X_test - means)  # / std

    # they are 2D originally in cifar
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    if noise_ratio > 0:
            n_samples = y_train.shape[0]
            n_noisy = int(noise_ratio*n_samples/100)
            noisy_idx = np.random.choice(n_samples, n_noisy, replace=False)
            for i in noisy_idx:
                y_train[i] = other_class(n_classes=NUM_CLASSES[dataset], current_class=y_train[i])
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    y_train = np_utils.to_categorical(y_train, NUM_CLASSES[dataset])
    y_test = np_utils.to_categorical(y_test, NUM_CLASSES[dataset])
    return X_train, X_test, y_train, y_test

In [6]:
final_result_list= []

In [7]:
for re in range(repeat):
    result_list = []
    for noise_ratio in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:          
        X_train, X_test, y_train, y_test = get_data(noise_ratio)    
        image_shape = X_train.shape[1:]
        model_quality = get_model(dataset, input_tensor=None, input_shape=image_shape, num_classes=NUM_CLASSES[dataset])
    
        optimizer = SGD(lr=0.01, decay=1e-4, momentum=0.9)
        model_quality.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
        datagen = ImageDataGenerator(
        featurewise_center = False,  # set input mean to 0 over the dataset
        samplewise_center = False,  # set each sample mean to 0
        featurewise_std_normalization = False,  # divide inputs by std of the dataset
        samplewise_std_normalization = False,  # divide each input by its std
        zca_whitening = False,  # apply ZCA whitening
        rotation_range = 0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range = 0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip = False,  # randomly flip images
        )
    
        datagen.fit(X_train)
        epochs_init = 100
        batch_size = 128
    # rand_repeat = 4  - X_train.shape[0] % 4
    # if rand_repeat != 4:
    #     chosen_repeat = np.random.choice(np.arange(0,X_train.shape[0]), rand_repeat, replace=False)
    #     X_train = np.append(X_train[chosen_repeat], X_train,axis=0)
    #     y_train = np.append(y_train[chosen_repeat], y_train,axis=0)

        h_quality  =  model_quality.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                            steps_per_epoch=X_train.shape[0]//batch_size, epochs=epochs_init,
                            validation_data=(X_test, y_test)
                            )
        result_list.append(h_quality.history['val_accuracy'][-1])
    print("result_list: ",result_list)
    final_result_list.append(result_list)



(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
20/20 [==============================] - 7s 362ms/step - loss: 7.6072 - accuracy: 0.2513 - val_loss: 7.8846 - val_accuracy: 0.1444
Epoch 2/100
20/20 [==============================] - 5s 226ms/step - loss: 6.4428 - accuracy: 0.5133 - val_loss: 6.7441 - val_accuracy: 0.3880
Epoch 3/100
20/20 [==============================] - 4s 223ms/step - loss: 5.7393 - accuracy: 0.6440 - val_loss: 6.2422 - val_accuracy: 0.4481
Epoch 4/100
20/20 [==============================] - 5s 227ms/step - loss: 5.1928 - accuracy: 0.7256 - val_loss: 5.8615 - val_accuracy: 0.4767
Epoch 5/100
20/20 [==============================] - 5s 228ms/step - loss: 4.7071 - accuracy: 0.7818 - val_loss: 5.6002 - val_accuracy: 0.4271
Epoch 6/100
20/20 [==============================] - 5s 226ms/step - loss: 4.3070 - accuracy: 0.8180 - val_loss: 5.3212 - val_accuracy: 0.4496
Epoch 7/100


Epoch 56/100
20/20 [==============================] - 4s 221ms/step - loss: 0.3326 - accuracy: 0.9881 - val_loss: 0.9076 - val_accuracy: 0.8451
Epoch 57/100
20/20 [==============================] - 4s 219ms/step - loss: 0.3215 - accuracy: 0.9892 - val_loss: 0.9082 - val_accuracy: 0.8391
Epoch 58/100
20/20 [==============================] - 4s 222ms/step - loss: 0.3123 - accuracy: 0.9881 - val_loss: 0.9049 - val_accuracy: 0.8180
Epoch 59/100
20/20 [==============================] - 4s 219ms/step - loss: 0.3049 - accuracy: 0.9873 - val_loss: 0.8111 - val_accuracy: 0.8556
Epoch 60/100
20/20 [==============================] - 4s 222ms/step - loss: 0.3088 - accuracy: 0.9833 - val_loss: 0.9179 - val_accuracy: 0.8211
Epoch 61/100
20/20 [==============================] - 4s 223ms/step - loss: 0.3001 - accuracy: 0.9869 - val_loss: 0.9521 - val_accuracy: 0.8105
Epoch 62/100
20/20 [==============================] - 4s 224ms/step - loss: 0.2858 - accuracy: 0.9885 - val_loss: 0.9344 - val_accuracy:

Epoch 69/100
20/20 [==============================] - 4s 225ms/step - loss: 0.5029 - accuracy: 0.9618 - val_loss: 1.5384 - val_accuracy: 0.7083
Epoch 70/100
20/20 [==============================] - 4s 222ms/step - loss: 0.4993 - accuracy: 0.9610 - val_loss: 1.3876 - val_accuracy: 0.7383
Epoch 71/100
20/20 [==============================] - 5s 226ms/step - loss: 0.4861 - accuracy: 0.9626 - val_loss: 1.5029 - val_accuracy: 0.7353
Epoch 72/100
20/20 [==============================] - 4s 221ms/step - loss: 0.4846 - accuracy: 0.9642 - val_loss: 1.4676 - val_accuracy: 0.7248
Epoch 73/100
20/20 [==============================] - 4s 224ms/step - loss: 0.4966 - accuracy: 0.9570 - val_loss: 1.4043 - val_accuracy: 0.7429
Epoch 74/100
20/20 [==============================] - 4s 224ms/step - loss: 0.4859 - accuracy: 0.9578 - val_loss: 1.2791 - val_accuracy: 0.7564
Epoch 75/100
20/20 [==============================] - 4s 221ms/step - loss: 0.5011 - accuracy: 0.9602 - val_loss: 1.3885 - val_accuracy:

Epoch 82/100
20/20 [==============================] - 5s 226ms/step - loss: 0.6685 - accuracy: 0.9442 - val_loss: 1.7784 - val_accuracy: 0.6647
Epoch 83/100
20/20 [==============================] - 5s 226ms/step - loss: 0.6723 - accuracy: 0.9438 - val_loss: 2.0263 - val_accuracy: 0.6030
Epoch 84/100
20/20 [==============================] - 4s 222ms/step - loss: 0.6323 - accuracy: 0.9514 - val_loss: 1.8071 - val_accuracy: 0.6541
Epoch 85/100
20/20 [==============================] - 4s 222ms/step - loss: 0.6521 - accuracy: 0.9434 - val_loss: 1.9031 - val_accuracy: 0.6466
Epoch 86/100
20/20 [==============================] - 4s 224ms/step - loss: 0.6391 - accuracy: 0.9458 - val_loss: 1.9782 - val_accuracy: 0.6226
Epoch 87/100
20/20 [==============================] - 4s 222ms/step - loss: 0.6657 - accuracy: 0.9387 - val_loss: 1.8337 - val_accuracy: 0.6331
Epoch 88/100
20/20 [==============================] - 4s 223ms/step - loss: 0.6453 - accuracy: 0.9438 - val_loss: 1.6911 - val_accuracy:

Epoch 95/100
20/20 [==============================] - 4s 218ms/step - loss: 0.7794 - accuracy: 0.9263 - val_loss: 2.2960 - val_accuracy: 0.5624
Epoch 96/100
20/20 [==============================] - 5s 227ms/step - loss: 0.7981 - accuracy: 0.9200 - val_loss: 2.2353 - val_accuracy: 0.5504
Epoch 97/100
20/20 [==============================] - 5s 236ms/step - loss: 0.7642 - accuracy: 0.9327 - val_loss: 2.2105 - val_accuracy: 0.5759
Epoch 98/100
20/20 [==============================] - 4s 220ms/step - loss: 0.7527 - accuracy: 0.9255 - val_loss: 2.3127 - val_accuracy: 0.5609
Epoch 99/100
20/20 [==============================] - 4s 222ms/step - loss: 0.7279 - accuracy: 0.9371 - val_loss: 2.3609 - val_accuracy: 0.5534
Epoch 100/100
20/20 [==============================] - 5s 226ms/step - loss: 0.7615 - accuracy: 0.9339 - val_loss: 2.1256 - val_accuracy: 0.5880
(2639, 128, 128, 3) (665, 128, 128, 3) (2639,) (665,)
Epoch 1/100
20/20 [==============================] - 5s 226ms/step - loss: 8.2061

...

Epoch 93/100
20/20 [==============================] - 4s 222ms/step - loss: 1.1621 - accuracy: 0.8873 - val_loss: 4.4506 - val_accuracy: 0.1895
Epoch 94/100
20/20 [==============================] - 4s 220ms/step - loss: 1.2026 - accuracy: 0.8734 - val_loss: 4.4653 - val_accuracy: 0.2226
Epoch 95/100
20/20 [==============================] - 5s 226ms/step - loss: 1.1900 - accuracy: 0.8809 - val_loss: 4.5534 - val_accuracy: 0.2105
Epoch 96/100
20/20 [==============================] - 4s 217ms/step - loss: 1.1738 - accuracy: 0.8785 - val_loss: 4.4675 - val_accuracy: 0.2406
Epoch 97/100
20/20 [==============================] - 4s 225ms/step - loss: 1.1720 - accuracy: 0.8801 - val_loss: 4.5301 - val_accuracy: 0.2391
Epoch 98/100
20/20 [==============================] - 5s 234ms/step - loss: 1.1838 - accuracy: 0.8809 - val_loss: 4.4869 - val_accuracy: 0.2466
Epoch 99/100
20/20 [==============================] - 5s 228ms/step - loss: 1.1847 - accuracy: 0.8746 - val_loss: 4.3804 - val_accuracy:

20/20 [==============================] - 4s 223ms/step - loss: 6.7346 - accuracy: 0.1286 - val_loss: 6.7356 - val_accuracy: 0.0466
Epoch 6/100
20/20 [==============================] - 5s 226ms/step - loss: 6.3868 - accuracy: 0.1637 - val_loss: 6.4914 - val_accuracy: 0.0526
Epoch 7/100
20/20 [==============================] - 4s 224ms/step - loss: 6.0624 - accuracy: 0.1816 - val_loss: 6.2667 - val_accuracy: 0.0586
Epoch 8/100
20/20 [==============================] - 5s 225ms/step - loss: 5.7752 - accuracy: 0.1916 - val_loss: 6.0271 - val_accuracy: 0.0556
Epoch 9/100
20/20 [==============================] - 4s 224ms/step - loss: 5.5094 - accuracy: 0.2059 - val_loss: 5.8715 - val_accuracy: 0.0421
Epoch 10/100
20/20 [==============================] - 4s 223ms/step - loss: 5.2640 - accuracy: 0.2461 - val_loss: 5.6655 - val_accuracy: 0.0466
Epoch 11/100
20/20 [==============================] - 4s 225ms/step - loss: 4.9991 - accuracy: 0.2644 - val_loss: 5.6084 - val_accuracy: 0.0496
Epoch 12/

20/20 [==============================] - 4s 221ms/step - loss: 1.4806 - accuracy: 0.8303 - val_loss: 5.8396 - val_accuracy: 0.0812
Epoch 63/100
20/20 [==============================] - 4s 219ms/step - loss: 1.4686 - accuracy: 0.8288 - val_loss: 5.9437 - val_accuracy: 0.0662
Epoch 64/100
20/20 [==============================] - 4s 223ms/step - loss: 1.4625 - accuracy: 0.8224 - val_loss: 5.7114 - val_accuracy: 0.0827
Epoch 65/100
20/20 [==============================] - 5s 228ms/step - loss: 1.4597 - accuracy: 0.8280 - val_loss: 5.7985 - val_accuracy: 0.0827
Epoch 66/100
20/20 [==============================] - 4s 219ms/step - loss: 1.4574 - accuracy: 0.8379 - val_loss: 5.8327 - val_accuracy: 0.0752
Epoch 67/100
20/20 [==============================] - 4s 224ms/step - loss: 1.4186 - accuracy: 0.8355 - val_loss: 5.7558 - val_accuracy: 0.0902
Epoch 68/100
20/20 [==============================] - 4s 224ms/step - loss: 1.3857 - accuracy: 0.8443 - val_loss: 5.8995 - val_accuracy: 0.0677
Epoch

Epoch 19/100
20/20 [==============================] - 4s 224ms/step - loss: 3.5326 - accuracy: 0.4691 - val_loss: 5.7288 - val_accuracy: 0.0211
Epoch 20/100
20/20 [==============================] - 4s 220ms/step - loss: 3.3897 - accuracy: 0.4855 - val_loss: 5.8013 - val_accuracy: 0.0241
Epoch 21/100
20/20 [==============================] - 4s 221ms/step - loss: 3.2816 - accuracy: 0.4974 - val_loss: 5.8170 - val_accuracy: 0.0331
Epoch 22/100
20/20 [==============================] - 4s 220ms/step - loss: 3.1943 - accuracy: 0.5195 - val_loss: 5.8363 - val_accuracy: 0.0211
Epoch 23/100
20/20 [==============================] - 4s 225ms/step - loss: 3.0256 - accuracy: 0.5380 - val_loss: 5.9865 - val_accuracy: 0.0195
Epoch 24/100
20/20 [==============================] - 4s 224ms/step - loss: 2.9608 - accuracy: 0.5639 - val_loss: 5.8241 - val_accuracy: 0.0316
Epoch 25/100
20/20 [==============================] - 4s 222ms/step - loss: 2.8381 - accuracy: 0.5814 - val_loss: 5.9764 - val_accuracy:

Epoch 76/100
20/20 [==============================] - 5s 227ms/step - loss: 1.3398 - accuracy: 0.8558 - val_loss: 6.9609 - val_accuracy: 0.0195
Epoch 77/100
20/20 [==============================] - 5s 227ms/step - loss: 1.3205 - accuracy: 0.8606 - val_loss: 7.2961 - val_accuracy: 0.0211
Epoch 78/100
20/20 [==============================] - 4s 217ms/step - loss: 1.2958 - accuracy: 0.8642 - val_loss: 7.2287 - val_accuracy: 0.0271
Epoch 79/100
20/20 [==============================] - 5s 226ms/step - loss: 1.3036 - accuracy: 0.8610 - val_loss: 7.3385 - val_accuracy: 0.0195
Epoch 80/100
20/20 [==============================] - 5s 226ms/step - loss: 1.3582 - accuracy: 0.8523 - val_loss: 7.3381 - val_accuracy: 0.0180
Epoch 81/100
20/20 [==============================] - 4s 222ms/step - loss: 1.2901 - accuracy: 0.8699 - val_loss: 7.8447 - val_accuracy: 0.0226
Epoch 82/100
20/20 [==============================] - 5s 226ms/step - loss: 1.2888 - accuracy: 0.8622 - val_loss: 7.4571 - val_accuracy:

In [11]:
final_result = np.array(final_result_list)

In [15]:
final_result

array([[0.82857144, 0.76240599, 0.66315788, 0.5879699 , 0.44661653,
        0.33383459, 0.28270677, 0.25263157, 0.13082707, 0.08120301,
        0.02556391],
       [0.83157897, 0.74586469, 0.66466165, 0.61052632, 0.49774435,
        0.36541355, 0.29473683, 0.21804512, 0.15789473, 0.08270676,
        0.01804511],
       [0.8511278 , 0.77443612, 0.66616541, 0.50977445, 0.45864663,
        0.39548871, 0.28421053, 0.24661654, 0.12631579, 0.07218045,
        0.03759398]])

In [14]:
np.mean(final_result,  axis = 0)

array([0.83709274, 0.76090227, 0.66466165, 0.56942356, 0.46766917,
       0.36491228, 0.28721804, 0.23909774, 0.13834586, 0.07869674,
       0.02706767])